In [7]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.linear_model import LinearRegression as LR
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import OLSInfluence
from scipy import stats
import matplotlib.pyplot as plt
import warnings
from statsmodels.stats.stattools import durbin_watson
from statsmodels.stats.outliers_influence import variance_inflation_factor
from scipy.stats import levene
from statsmodels.formula.api import ols


warnings.filterwarnings('ignore')

df=pd.read_excel('NutritionStudy.xls')
df

,ID,Age,Smoke,Quetelet,Calories,Fat,Fiber,Alcohol,Cholesterol,BetaDiet,RetinolDiet,BetaPlasma,RetinolPlasma,Gender,VitaminUse,PriorSmoke
0,1,64,No,21.4838,1298.8,57.0,6.3,0.0,170.3,1945,890,200,915,Female,Regular,2
1,2,76,No,23.8763,1032.5,50.1,15.8,0.0,75.8,2653,451,124,727,Female,Regular,1
2,3,38,No,20.0108,2372.3,83.6,19.1,14.1,257.9,6321,660,328,721,Female,Occasional,2
3,4,40,No,25.1406,2449.5,97.5,26.5,0.5,332.6,1061,864,153,615,Female,No,2
4,5,72,No,20.9850,1952.1,82.6,16.2,0.0,170.8,2863,1209,92,799,Female,Regular,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
310,311,46,No,25.8967,2263.6,98.2,19.4,2.6,306.5,2572,1261,164,216,Female,No,2
311,312,45,No,23.8270,1841.1,84.2,14.1,2.2,257.7,1665,465,80,328,Female,Regular,1
312,313,49,No,24.2613,1125.6,44.8,11.9,4.0,150.5,6943,520,300,502,Female,Regular,1
313,314,31,No,23.4525,2729.6,144.4,13.2,2.2,381.8,741,644,121,684,Female,Regular,2


# Task 1

In [8]:
desc_stats = df.groupby('PriorSmoke')['Cholesterol'].describe()
print(desc_stats)

            count        mean         std   min    25%    50%     75%    max
PriorSmoke                                                                  
1           157.0  228.391083  134.228448  37.7  141.2  195.8  271.80  900.7
2           115.0  250.424348  121.691586  46.3  160.4  216.7  340.55  747.5
3            43.0  272.532558  145.919098  78.3  174.3  239.2  343.25  718.8


In [9]:
model = ols('Cholesterol ~ C(PriorSmoke)', data=df).fit()
anova_table = sm.stats.anova_lm(model, typ=2)
print(anova_table)

                     sum_sq     df        F    PR(>F)
C(PriorSmoke)  7.725768e+04    2.0  2.23471  0.108734
Residual       5.393183e+06  312.0      NaN       NaN


The descriptive statistics you provided show the breakdown of Cholesterol by the PriorSmoke variable. Here's an interpretation of these results:

Group 1 (PriorSmoke = 1): The average Cholesterol level is approximately 228.39, with a standard deviation of 134.23. The Cholesterol levels in this group range from a minimum of 37.7 to a maximum of 900.7.
Group 2 (PriorSmoke = 2): The average Cholesterol level is approximately 250.42, with a standard deviation of 121.69. The Cholesterol levels in this group range from 46.3 to 747.5.
Group 3 (PriorSmoke = 3): The average Cholesterol level is approximately 272.53, with a standard deviation of 145.92. The Cholesterol levels in this group range from 78.3 to 718.8.
From these statistics, we can see that the mean Cholesterol level increases with the PriorSmoke category, suggesting a trend between smoking history and Cholesterol levels. However, the descriptive statistics alone don't provide statistical evidence for this trend.

To ascertain whether these differences are statistically significant, we would need to perform an ANOVA test, as outlined earlier. The result of that ANOVA would give us a p-value that we could use to determine whether the differences in means are statistically significant.

If you can provide the output of the ANOVA, we can further interpret those results!